<a href="https://colab.research.google.com/github/ParthaSarathiChakraborty/KaggleRepository/blob/main/Databricks_Predict_Ads_Clicks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%scala

val csv = spark.read.option("inferSchema","true").option("header", "true").csv("/FileStore/tables/advertising-1.csv")

csv.show()


In [ ]:
%scala

csv.printSchema()

In [ ]:
-- Statistics of Data


%scala

csv.select("DailyTimeSpentonSite", "Age", "AreaIncome", "DailyInternetUsage", "AdTopicLine", "City", "Male", "Country", "Timestamp", "ClickedonAd" ).describe().show()


In [ ]:
-- Create Temporary View so we can perform Spark SQL on Data

%scala

csv.createOrReplaceTempView("AdsData");

In [ ]:
-- One Visualization to Rule Them All​

%sql

select * from AdsData

In [ ]:
-- Collecting all String Columns into an Array

%scala

var StringfeatureCol = Array("AdTopicLine", "City", "Country", "Timestamp");

In [ ]:
-- Example of StringIndexer

%scala

import org.apache.spark.ml.feature.StringIndexer

val df = spark.createDataFrame(
  Seq((0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c"))
).toDF("id", "category")

df.show()

val indexer = new StringIndexer()
  .setInputCol("category")
  .setOutputCol("categoryIndex")

val indexed = indexer.fit(df).transform(df)

indexed.show()


In [ ]:
/*
Define the Pipeline​
A predictive model often requires multiple stages of feature preparation.

A pipeline consists of a series of transformer and estimator stages that typically prepare a DataFrame for modeling and then train a predictive model.

In this case, you will create a pipeline with stages:

A StringIndexer estimator that converts string values to indexes for categorical features
A VectorAssembler that combines categorical features into a single vector
*/


%scala

import org.apache.spark.ml.attribute.Attribute
import org.apache.spark.ml.feature.{IndexToString, StringIndexer}
import org.apache.spark.ml.{Pipeline, PipelineModel}

val indexers = StringfeatureCol.map { colName =>
  new StringIndexer().setInputCol(colName).setHandleInvalid("skip").setOutputCol(colName + "_indexed")
}

val pipeline = new Pipeline()
                    .setStages(indexers)      

val AdsFinalDF = pipeline.fit(csv).transform(csv)

In [ ]:
-- Print Schema to view String Columns are converted in to equivalent Numerical Columns

%scala

AdsFinalDF.printSchema()

In [ ]:
/*
Split the Data
It is common practice when building machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. 
In this project, you will use 70% of the data for training, and reserve 30% for testing.
*/

%scala

val splits = AdsFinalDF.randomSplit(Array(0.7, 0.3))
val train = splits(0)
val test = splits(1)
val train_rows = train.count()
val test_rows = test.count()
println("Training Rows: " + train_rows + " Testing Rows: " + test_rows)

In [ ]:
-- Vector assembler 


%scala

import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().setInputCols(Array("DailyTimeSpentonSite", "Age", "AreaIncome", "DailyInternetUsage", "AdTopicLine_indexed", "City_indexed", "Male", "Country_indexed", "Timestamp_indexed")).setOutputCol("features")

val training = assembler.transform(train).select($"features", $"ClickedonAd".alias("label"))

training.show(false)

In [ ]:
/*
Train a Classification Model
Next, you need to train a Classification model using the training data. To do this, create an instance of the LogisticRegression algorithm you want to use and 
use its fit method to train a model based on the training DataFrame. In this project, 
you will use a Logistic Regression Classifier algorithm – though you can use the same technique for any of the regression algorithms supported in the spark.ml API
*/

%scala
import org.apache.spark.ml.classification.LogisticRegression

val lr = new LogisticRegression().setLabelCol("label").setFeaturesCol("features").setMaxIter(10).setRegParam(0.3)
val model = lr.fit(training)
println ("Model trained!")

In [ ]:
-- Prepare the Testing Data

%scala

val testing = assembler.transform(test).select($"features", $"ClickedonAd".alias("trueLabel"))
testing.show(false)

In [ ]:
-- Test the Model

%scala

val prediction = model.transform(testing)
val predicted = prediction.select("features", "prediction", "probability", "trueLabel")
predicted.show()

In [ ]:
-- Evaluating a Model (We got 93% Accuracy)


%scala

import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator


val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("trueLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(prediction)

Output:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = MulticlassClassificationEvaluator: uid=mcEval_ebd948da765b, metricName=accuracy, metricLabel=0.0, beta=1.0, eps=1.0E-15
accuracy: Double = 0.9377049180327869


In [ ]:
val tp = predicted.filter("prediction == 1 AND truelabel == 1").count().toFloat
val fp = predicted.filter("prediction == 1 AND truelabel == 0").count().toFloat
val tn = predicted.filter("prediction == 0 AND truelabel == 0").count().toFloat
val fn = predicted.filter("prediction == 0 AND truelabel == 1").count().toFloat
val metrics = spark.createDataFrame(Seq(
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn)))).toDF("metric", "value")
metrics.show()